In [1]:
import pandas as pd
import numpy as np
import os
from data.contracts import contract_addresses, stkAAVE_contract_addresses, xSUSHI_contract_addresses

transfers_cols = ['DateTime','From','To','Quantity','Method']

In [2]:
## aUSDC Aave interest bearing USDC
# Holders, filter out contracts
aUSDC_holders = pd.read_csv('data/aUSDC/aUSDC_holders.csv').sort_values(by='Balance', ascending=False)
aUSDC_holders = aUSDC_holders[~aUSDC_holders['HolderAddress'].isin(contract_addresses)].drop([1259]).set_index('HolderAddress')
aUSDC_holders = pd.cut(aUSDC_holders['Balance'], bins=[0,300,3000,30000,300000,3000000,30000000,300000000], labels=[1,2,3,4,5,6,7]).to_frame().rename(columns={'Balance': 'aUSDC_balance'}).astype('float') # Discretize


# Transfers, filter out contracts, keep only transactions that mint/burn aUSDC
aUSDC_transfers = pd.DataFrame()
for f in os.listdir('data/aUSDC/'):
    if f.endswith('.csv') and (f[0] in '0123'):
        data = pd.read_csv('data/aUSDC/'+f, usecols=transfers_cols)
        if data.loc[0]['DateTime'].startswith('2021'):
            data['DateTime'] = pd.to_datetime(data['DateTime'], format='%Y-%m-%d %H:%M:%S')
        else:
            data['DateTime'] = pd.to_datetime(data['DateTime'], format='%d/%m/%Y %H:%M')
        if len(aUSDC_transfers):
            aUSDC_transfers = pd.concat([aUSDC_transfers,data])
        else:
            aUSDC_transfers = data
            
flash_loan_addresses = list(set(list(aUSDC_transfers[aUSDC_transfers['Method']=='Flash Loan']['From'].unique()) + list(aUSDC_transfers[aUSDC_transfers['Method']=='Flash Loan']['To'].unique())))
flash_loan_addresses.remove('0x0000000000000000000000000000000000000000')
aUSDC_transfers = aUSDC_transfers[(~aUSDC_transfers['From'].isin(contract_addresses))&(~aUSDC_transfers['To'].isin(contract_addresses))]
aUSDC_transfers = aUSDC_transfers[(aUSDC_transfers['From']=='0x0000000000000000000000000000000000000000')|(aUSDC_transfers['To']=='0x0000000000000000000000000000000000000000')].reset_index(drop=True)
aUSDC_transfers['Quantity'] = aUSDC_transfers['Quantity'].apply(lambda x: float(''.join(x.split(','))))


# Agg
from__ = aUSDC_transfers[aUSDC_transfers['From']!='0x0000000000000000000000000000000000000000'].rename(columns={"From": "Address"}).drop(columns='To')
to__ = aUSDC_transfers[aUSDC_transfers['To']!='0x0000000000000000000000000000000000000000'].rename(columns={"To": "Address"}).drop(columns='From')
combined_aUSDC_transfers = pd.concat([from__, to__])
latest_day = combined_aUSDC_transfers['DateTime'].max()
# 30-day number of transactions
combined_aUSDC_transfers_30 = combined_aUSDC_transfers[combined_aUSDC_transfers['DateTime']>=(latest_day-pd.Timedelta(days=30))]
num_transactions_30 = np.log10(combined_aUSDC_transfers_30['Address'].value_counts()).to_frame().rename(columns={'Address': '30_day_log10_aUSDC_num_transactions'})
# 90-day number of transactions
combined_aUSDC_transfers_90 = combined_aUSDC_transfers[combined_aUSDC_transfers['DateTime']>=(latest_day-pd.Timedelta(days=90))]
num_transactions_90 = np.log10(combined_aUSDC_transfers_90['Address'].value_counts()).to_frame().rename(columns={'Address': '90_day_log10_aUSDC_num_transactions'})
# 180-day number of transactions
combined_aUSDC_transfers_180 = combined_aUSDC_transfers[combined_aUSDC_transfers['DateTime']>=(latest_day-pd.Timedelta(days=180))]
num_transactions_180 = np.log10(combined_aUSDC_transfers_180['Address'].value_counts()).to_frame().rename(columns={'Address': '180_day_log10_aUSDC_num_transactions'})
# 180-day average size of transaction
avg_size_180 = np.log10(combined_aUSDC_transfers_180[['Address','Quantity']].groupby('Address').aggregate('mean')).rename(columns={'Quantity': 'log10_aUSDC_avg_size'})


# Feature engineering
to_ = pd.DataFrame(aUSDC_transfers['To'].value_counts())
from_ = pd.DataFrame(aUSDC_transfers['From'].value_counts())
features = to_.join(from_, how='outer').rename(columns={"To": "num_USDC_deposits", "From": "num_USDC_withdrawals"}).fillna(0).drop(labels='0x0000000000000000000000000000000000000000')
features = features.join([aUSDC_holders,num_transactions_30,num_transactions_90,num_transactions_180,avg_size_180], how='left').fillna(0)

In [3]:
## variableDebtUSDC Aave variable debt bearing USDC
# Holders, filter out contracts
variableDebtUSDC_holders = pd.read_csv('data/variableDebtUSDC/variableDebtUSDC_holders.csv').sort_values(by='Balance', ascending=False)
variableDebtUSDC_holders = variableDebtUSDC_holders[~variableDebtUSDC_holders['HolderAddress'].isin(contract_addresses)].drop([696]).set_index('HolderAddress')
variableDebtUSDC_holders = pd.cut(variableDebtUSDC_holders['Balance'], bins=[0,1000,10000,100000,1000000,10000000,300000000], labels=[1,2,3,4,5,6]).to_frame().rename(columns={'Balance': 'variableDebtUSDC_balance'}).astype('float') # Discretize


# Transfers, filter out contracts, keep only transactions that mint/burn variableDebtUSDC
variableDebtUSDC_transfers = pd.DataFrame()
for f in os.listdir('data/variableDebtUSDC/'):
    if f.endswith('.csv') and (f[0] in '0123'):
        data = pd.read_csv('data/variableDebtUSDC/'+f, usecols=transfers_cols)
        if data.loc[0]['DateTime'].startswith('2021'):
            data['DateTime'] = pd.to_datetime(data['DateTime'], format='%Y-%m-%d %H:%M:%S')
        else:
            data['DateTime'] = pd.to_datetime(data['DateTime'], format='%d/%m/%Y %H:%M')
        if len(variableDebtUSDC_transfers):
            variableDebtUSDC_transfers = pd.concat([variableDebtUSDC_transfers,data])
        else:
            variableDebtUSDC_transfers = data
            
flash_loan_addresses2 = list(set(list(variableDebtUSDC_transfers[variableDebtUSDC_transfers['Method']=='Flash Loan']['From'].unique()) + list(variableDebtUSDC_transfers[variableDebtUSDC_transfers['Method']=='Flash Loan']['To'].unique())))
flash_loan_addresses2.remove('0x0000000000000000000000000000000000000000')
variableDebtUSDC_transfers = variableDebtUSDC_transfers[(~variableDebtUSDC_transfers['From'].isin(contract_addresses))&(~variableDebtUSDC_transfers['To'].isin(contract_addresses))]
variableDebtUSDC_transfers = variableDebtUSDC_transfers[(variableDebtUSDC_transfers['From']=='0x0000000000000000000000000000000000000000')|(variableDebtUSDC_transfers['To']=='0x0000000000000000000000000000000000000000')].reset_index(drop=True)
variableDebtUSDC_transfers['Quantity'] = variableDebtUSDC_transfers['Quantity'].apply(lambda x: float(''.join(x.split(','))))


# Agg
from__ = variableDebtUSDC_transfers[variableDebtUSDC_transfers['From']!='0x0000000000000000000000000000000000000000'].rename(columns={"From": "Address"}).drop(columns='To')
to__ = variableDebtUSDC_transfers[variableDebtUSDC_transfers['To']!='0x0000000000000000000000000000000000000000'].rename(columns={"To": "Address"}).drop(columns='From')
combined_variableDebtUSDC_transfers = pd.concat([from__, to__])
latest_day = combined_variableDebtUSDC_transfers['DateTime'].max()
# 30-day number of transactions
combined_variableDebtUSDC_transfers_30 = combined_variableDebtUSDC_transfers[combined_variableDebtUSDC_transfers['DateTime']>=(latest_day-pd.Timedelta(days=30))]
num_transactions_30 = np.log10(combined_variableDebtUSDC_transfers_30['Address'].value_counts()).to_frame().rename(columns={'Address': '30_day_log10_variableDebtUSDC_num_transactions'})
# 90-day number of transactions
combined_variableDebtUSDC_transfers_90 = combined_variableDebtUSDC_transfers[combined_variableDebtUSDC_transfers['DateTime']>=(latest_day-pd.Timedelta(days=90))]
num_transactions_90 = np.log10(combined_variableDebtUSDC_transfers_90['Address'].value_counts()).to_frame().rename(columns={'Address': '90_day_log10_variableDebtUSDC_num_transactions'})
# 180-day number of transactions
combined_variableDebtUSDC_transfers_180 = combined_variableDebtUSDC_transfers[combined_variableDebtUSDC_transfers['DateTime']>=(latest_day-pd.Timedelta(days=180))]
num_transactions_180 = np.log10(combined_variableDebtUSDC_transfers_180['Address'].value_counts()).to_frame().rename(columns={'Address': '180_day_log10_variableDebtUSDC_num_transactions'})
# 180-day average size of transaction
avg_size_180 = np.log10(combined_variableDebtUSDC_transfers_180[['Address','Quantity']].groupby('Address').aggregate('mean')).rename(columns={'Quantity': 'log10_variableDebtUSDC_avg_size'})


# Feature engineering
to_ = pd.DataFrame(variableDebtUSDC_transfers['To'].value_counts())
from_ = pd.DataFrame(variableDebtUSDC_transfers['From'].value_counts())
features2 = to_.join(from_, how='outer').rename(columns={"To": "num_USDC_borrows", "From": "num_USDC_repayments"}).fillna(0).drop(labels='0x0000000000000000000000000000000000000000')
features2 = features2.join([variableDebtUSDC_holders,num_transactions_30,num_transactions_90,num_transactions_180,avg_size_180], how='left').fillna(0)
features = features.join(features2, how='outer').fillna(0)
features['flash_loan'] = [True if (i in flash_loan_addresses) or (i in flash_loan_addresses2) else False for i in features.index]

In [4]:
## stkAAVE
# Holders, filter out contracts
stkAAVE_holders = pd.read_csv('data/stkAAVE/stkAAVE_holders.csv').sort_values(by='Balance', ascending=False)
stkAAVE_holders = stkAAVE_holders[~stkAAVE_holders['HolderAddress'].isin(stkAAVE_contract_addresses)].set_index('HolderAddress')
stkAAVE_holders = pd.cut(stkAAVE_holders['Balance'], bins=[0,10,100,1000,10000,100000,1000000], labels=[1,2,3,4,5,6]).to_frame().rename(columns={'Balance': 'stkAAVE_balance'}).astype('float') # Discretize

features = features.join([stkAAVE_holders], how='left').fillna(0)
features['stkAAVE'] = features['stkAAVE_balance'].apply(lambda x: True if x else False)

In [5]:
## xSUSHI
# Holders, filter out contracts
xSUSHI_holders = pd.read_csv('data/xSUSHI/xSUSHI_holders.csv').sort_values(by='Balance', ascending=False)
xSUSHI_holders = xSUSHI_holders[~xSUSHI_holders['HolderAddress'].isin(xSUSHI_contract_addresses)].set_index('HolderAddress')
xSUSHI_holders = pd.cut(xSUSHI_holders['Balance'], bins=[0,30,300,3000,30000,300000,3000000,30000000], labels=[1,2,3,4,5,6,7]).to_frame().rename(columns={'Balance': 'xSUSHI_balance'}).astype('float') # Discretize

features = features.join([xSUSHI_holders], how='left').fillna(0)
features['xSUSHI'] = features['xSUSHI_balance'].apply(lambda x: True if x else False)

In [6]:
## Aave governance voters
gov = pd.read_csv('data/AaveGovernanceV2.csv', index_col=False, usecols=['From','Method'])
gov = gov[gov['Method']=='Submit Vote'][['From']].value_counts().to_frame().rename(columns={0: 'num_aave_gov_votes'})

features = features.join([gov], how='left').fillna(0)
features['aave_voter'] = features['num_aave_gov_votes'].apply(lambda x: True if x else False)

In [7]:
## AAVE liqduidity mining
claim_rewards = pd.read_csv('data/LiquidityMining.csv', index_col=False, usecols=['Txhash','DateTime','From','Method'])
claim_rewards = claim_rewards[claim_rewards['Method']=='Claim Rewards'].set_index('Txhash')
claim_rewards['DateTime'] = pd.to_datetime(claim_rewards['DateTime'], format='%Y-%m-%d %H:%M:%S')
aave_rewards = pd.read_csv('data/LiquidityMiningAAVE.csv', index_col=False, usecols=['Txhash','Value']).set_index('Txhash')
claim_rewards = claim_rewards.join([aave_rewards], how='left')[['From','DateTime','Value']].dropna()

total_rewards = claim_rewards[['From','Value']].groupby('From').aggregate('sum').rename(columns={'Value': 'total_aave_rewards'})
total_claims = np.log(claim_rewards[['From','DateTime']].groupby('From').aggregate('count').rename(columns={'DateTime': 'total_claim_actions'}))

features = features.join([total_rewards], how='left').fillna(0)
features = features.join([total_claims], how='left').fillna(0)

In [11]:
features

,num_USDC_deposits,num_USDC_withdrawals,aUSDC_balance,30_day_log10_aUSDC_num_transactions,90_day_log10_aUSDC_num_transactions,180_day_log10_aUSDC_num_transactions,log10_aUSDC_avg_size,num_USDC_borrows,num_USDC_repayments,variableDebtUSDC_balance,...,log10_variableDebtUSDC_avg_size,flash_loan,stkAAVE_balance,stkAAVE,xSUSHI_balance,xSUSHI,num_aave_gov_votes,aave_voter,total_aave_rewards,total_claim_actions
0x00000000000cd56832ce5dfbcbff02e7ec639bc9,2.0,2.0,0.0,0.0,0.60206,0.60206,5.005052,0.0,0.0,0.0,...,0.000000,False,0.0,False,0.0,False,0.0,False,0,0.0
0x000000003ce0cf2c037493b1dc087204bd7f713e,1.0,0.0,1.0,0.0,0.00000,0.00000,4.305872,0.0,0.0,0.0,...,0.000000,False,0.0,False,0.0,False,0.0,False,0,0.0
0x0000000484f2217f1a64eb6d24b5cee446faeae5,1.0,1.0,0.0,0.0,0.00000,0.30103,6.024089,0.0,0.0,0.0,...,0.000000,False,0.0,False,0.0,False,0.0,False,0,0.0
0x000000aaee6a496aaf7b7452518781786313400f,0.0,4.0,1.0,0.0,0.00000,0.60206,5.016485,0.0,0.0,0.0,...,0.000000,False,0.0,False,0.0,False,0.0,False,0,0.0
0x000018bbb8df8de9e3eaf772db1c4eec228ef06c,1.0,0.0,1.0,0.0,0.00000,0.00000,1.477355,0.0,0.0,0.0,...,0.000000,False,0.0,False,0.0,False,0.0,False,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0xffe5332949efdf2b389597b8eb33da055bd9a877,0.0,0.0,0.0,0.0,0.00000,0.00000,0.000000,1.0,1.0,3.0,...,4.521089,False,0.0,False,0.0,False,0.0,False,0,0.0
0xffe6212baf6c88850dcd6511cd32be11c50d3a61,0.0,0.0,0.0,0.0,0.00000,0.00000,0.000000,2.0,0.0,4.0,...,5.207127,False,0.0,False,0.0,False,0.0,False,0,0.0
0xffeb122a31cd9bba428adcc1427c33dfca3aeb10,1.0,1.0,0.0,0.0,0.00000,0.30103,4.528979,0.0,0.0,0.0,...,0.000000,False,0.0,False,0.0,False,0.0,False,0,0.0
0xffefdcfff613c9bbb9928f6ff44f07c7b562bfdf,1.0,1.0,0.0,0.0,0.30103,0.30103,4.097385,0.0,0.0,0.0,...,0.000000,False,0.0,False,0.0,False,0.0,False,0.189423619,0.0
